In [115]:
import pandas as pd

In [116]:
used_columns=['permit_type',
              'review_type',
              'application_start_date',
              'processing_time',
#              'building_fee_paid',
#              'zoning_fee_paid',
#              'other_fee_paid',
#              'subtotal_paid',
#              'building_fee_unpaid',
#              'zoning_fee_unpaid',
#              'other_fee_unpaid',
#              'subtotal_unpaid',
#              'building_fee_waived',
#              'zoning_fee_waived',
#              'other_fee_waived',
#              'subtotal_waived',
              'total_fee',
              'reported_cost',
              'community_area',
#              'census_tract',
#              'ward',
              'latitude',
              'longitude',
              'location'
             ]

# Fetch small data set
permit_df = pd.read_json("https://data.cityofchicago.org/resource/building-permits.json")
# Store small data set in csv
permit_df.to_csv('building-permits-small.csv', index=False)
# Open csv and only keep useful columns
permit_df= pd.read_csv('building-permits-small.csv', usecols=used_columns)


In [117]:
permit_df.head()

,permit_type,review_type,application_start_date,processing_time,total_fee,reported_cost,community_area,latitude,longitude,location
0,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2007-10-25T00:00:00.000,0,65.0,0.0,NaN,NaN,NaN,NaN
1,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2010-07-28T00:00:00.000,0,50.0,1000.0,30.0,NaN,NaN,NaN
2,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2020-01-08T00:00:00.000,0,1500.0,1000.0,32.0,41.879597,-87.633615,"{'type': 'Point', 'coordinates': [-87.63361452..."
3,PERMIT - ELEVATOR EQUIPMENT,CONVEYANCE DEVICE PERMIT,2020-01-08T00:00:00.000,0,150.0,20912.0,75.0,41.678426,-87.684266,"{'type': 'Point', 'coordinates': [-87.68426581..."
4,PERMIT - SIGNS,SIGN PERMIT,2019-03-29T00:00:00.000,108,400.0,7500.0,32.0,41.878543,-87.632121,"{'type': 'Point', 'coordinates': [-87.63212057..."


## Data cleaning

In [118]:
# How many permits are there?

permit_df_shape=permit_df.shape

print(f"There are {permit_df_shape[0]} total permits in data set.")

# Remove any rows with empty application_start_date

permit_df = permit_df[pd.notnull(permit_df['application_start_date'])]

permit_df_shape=permit_df.shape

print(f"There are {permit_df_shape[0]} total permits in data with nonempty start date.")

# Remove any rows with empty community_area and convert to integers

permit_df = permit_df[pd.notnull(permit_df['community_area'])]

permit_df['community_area']=permit_df['community_area'].astype('int')

permit_df_shape=permit_df.shape

print(f"There are {permit_df_shape[0]} total permits in data with nonempty start date and nonempyt community area.")

# Fixing date format

permit_df['application_start_date']=pd.to_datetime(permit_df['application_start_date'])

# Dictionary for later use

community_areas_dict = {}
with open('data/community_areas_dict') as f:
    for line in f:
        (key, val) = line.rstrip("\n").split(";")
        community_areas_dict[int(key)] = val
        
permit_df.head()

There are 1000 total permits in data set.
There are 1000 total permits in data with nonempty start date.
There are 969 total permits in data with nonempty start date and nonempyt community area.


,permit_type,review_type,application_start_date,processing_time,total_fee,reported_cost,community_area,latitude,longitude,location
1,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2010-07-28,0,50.0,1000.0,30,NaN,NaN,NaN
2,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2020-01-08,0,1500.0,1000.0,32,41.879597,-87.633615,"{'type': 'Point', 'coordinates': [-87.63361452..."
3,PERMIT - ELEVATOR EQUIPMENT,CONVEYANCE DEVICE PERMIT,2020-01-08,0,150.0,20912.0,75,41.678426,-87.684266,"{'type': 'Point', 'coordinates': [-87.68426581..."
4,PERMIT - SIGNS,SIGN PERMIT,2019-03-29,108,400.0,7500.0,32,41.878543,-87.632121,"{'type': 'Point', 'coordinates': [-87.63212057..."
5,PERMIT - ELECTRIC WIRING,EASY PERMIT WEB,2019-10-03,0,450.0,100.0,28,41.886739,-87.660158,"{'type': 'Point', 'coordinates': [-87.66015770..."


In [119]:
# How many sign are there?
sign_permit_df=permit_df[permit_df['permit_type']=='PERMIT - SIGNS']
print(f'There are {sign_permit_df.shape[0]} sign permits')
# Remove sign permits
#permit_df=permit_df[permit_df['permit_type']!='PERMIT - SIGNS']
# How many EASY PERMIT WEB web permits are there?
easy_permit_df=permit_df[permit_df['review_type']=='EASY PERMIT WEB']
print(f'There are {easy_permit_df.shape[0]} easy web permits')
# Remove easy permits
#permit_df=permit_df[permit_df['review_type']!='EASY PERMIT WEB']

There are 59 sign permits
There are 176 easy web permits


## First Objective: How does start date affect length to approval?

In [120]:
# Extract month from dates

permit_df['start_month'] = permit_df['application_start_date'].dt.month

In [121]:
month_proc_time_gb=permit_df[['start_month','processing_time']].groupby(['start_month'], as_index=True)

month_proc_time_df=pd.DataFrame(month_proc_time_gb.mean())

month_proc_time_df=month_proc_time_df.rename(columns={"processing_time": "ave_proc_time"})

month_proc_time_df

,ave_proc_time
start_month,
1,19.885246
2,34.107143
3,28.254237
4,26.750000
5,47.000000
6,94.014085
7,8.880137
8,17.553191
9,61.377778


In [122]:
month_num_applications_df=pd.DataFrame(month_proc_time_gb.count())

month_num_applications_df=month_num_applications_df.rename(columns={"processing_time": "num_of_applications"})

month_num_applications_df

,num_of_applications
start_month,
1,61
2,28
3,59
4,52
5,49
6,71
7,292
8,47
9,45


In [123]:
# Redo analysis without instantly approved applications?



## Second Objective: How does community area affect length to approval?

In [124]:
com_ar_proc_time_gb=permit_df[['community_area','processing_time','total_fee']].groupby(['community_area'], as_index=False)

com_ar_proc_time_df=pd.DataFrame(com_ar_proc_time_gb.mean())

com_ar_proc_time_df=com_ar_proc_time_df.replace({"community_area": community_areas_dict})

com_ar_proc_time_df

,community_area,processing_time,total_fee
0,Rogers Park,24.200000,252.900000
1,West Ridge,8.428571,441.428571
2,Uptown,2.571429,560.714286
3,Lincoln Square,50.866667,438.648000
4,North Center,24.307692,4911.700000
...,...,...,...
70,Washington Heights,0.285714,410.714286
71,Mount Greenwood,0.000000,550.000000
72,Morgan Park,0.666667,556.853333
73,Ohare,6.666667,12249.250000


In [125]:
# iterating the columns 
for col in permit_df.columns: 
    print(col) 

permit_type
review_type
application_start_date
processing_time
total_fee
reported_cost
community_area
latitude
longitude
location
start_month


In [126]:
permit_df['community_area']

1      30
2      32
3      75
4      32
5      28
       ..
995    65
996    74
997    23
998    61
999    31
Name: community_area, Length: 969, dtype: int32

## Objective 3 How do permits per population in a community area compare for diffenet community areas.  

In [127]:
# Data from https://datahub.cmap.illinois.gov/dataset/2010-census-data-summarized-to-chicago-community-areas

# File to Load (Remember to Change These)
file_to_load = "data/CCASF12010CMAP.csv"

# Read Purchasing File and store into Pandas dataframe
population_data = pd.read_csv(file_to_load)
population_data.head()

,GEOGNAME,GEOGKEYX,P0050001,P0050003,P0050004,P0050005,P0050006,P0050007,P0050008,P0050009,...,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128
0,Geog,GeogKey,Total Population,"Not Hispanic or Latino, White alone","Not Hispanic or Latino, Black or African Ameri...","Not Hispanic or Latino, American Indian and Al...","Not Hispanic or Latino, Asian alone","Not Hispanic or Latino, Native Hawaiian and Ot...","Not Hispanic or Latino, Some Other Race alone","Not Hispanic or Latino, Two or More Races",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Rogers Park,1,54991,21618,14461,142,3523,39,202,1573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,West Ridge,2,71942,30706,8015,189,16184,3,225,1919,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Uptown,3,56362,29098,11275,191,6414,10,130,1235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Lincoln Square,4,39493,24916,1488,72,4399,11,128,917,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
com_ar_ct_df=pd.DataFrame(com_ar_proc_time_gb.count())

com_ar_ct_df['Total Population']=''
for j in range(0,com_ar_ct_df.shape[0]):
    com_ar_ct_df['Total Population'].iloc[j]=population_data['P0050001'].iloc[int(com_ar_ct_df['community_area'].iloc[j])]
com_ar_ct_df['Total Population']=pd.to_numeric(com_ar_ct_df['Total Population'])

com_ar_ct_df['perm_p_person']=(com_ar_ct_df['processing_time']/com_ar_ct_df['Total Population'])

# Apply community area labels
com_ar_ct_df=com_ar_ct_df.replace({"community_area": community_areas_dict})

com_ar_ct_df

,community_area,processing_time,total_fee,Total Population,perm_p_person
0,Rogers Park,10,10,54991,0.000182
1,West Ridge,14,14,71942,0.000195
2,Uptown,7,7,56362,0.000124
3,Lincoln Square,15,15,39493,0.000380
4,North Center,13,13,31867,0.000408
...,...,...,...,...,...
70,Washington Heights,7,7,26493,0.000264
71,Mount Greenwood,3,3,19093,0.000157
72,Morgan Park,6,6,22544,0.000266
73,Ohare,6,6,12756,0.000470


## Object 4 What is the average permit fee for all projects in a community area?

In [169]:
com_ar_fee_df=pd.DataFrame(com_ar_proc_time_gb.mean())

# Apply community area labels
com_ar_fee_df=com_ar_fee_df.replace({"community_area": community_areas_dict})

com_ar_fee_df

,community_area,processing_time,total_fee
0,Rogers Park,24.200000,252.900000
1,West Ridge,8.428571,441.428571
2,Uptown,2.571429,560.714286
3,Lincoln Square,50.866667,438.648000
4,North Center,24.307692,4911.700000
...,...,...,...
70,Washington Heights,0.285714,410.714286
71,Mount Greenwood,0.000000,550.000000
72,Morgan Park,0.666667,556.853333
73,Ohare,6.666667,12249.250000


## Objective 5 Find highest and lowest income Community area.  What is the most common permit in these community areas?

In [130]:
# Data from https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2/data

# File to Load (Remember to Change These)
file_to_load = "data/Per_Capita_Income.csv"

# Read Purchasing File and store into Pandas dataframe
income_data = pd.read_csv(file_to_load)
income_data.head()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0


## What location paid the most permit fees?  (in each community area)

In [170]:
com_area_indexes=pd.to_numeric(permit_df['community_area'].dropna()).tolist()
com_area_indexes=[int(index) for index in com_area_indexes]

max_indicies=[permit_df[(permit_df['community_area']==index)]['total_fee'].idxmax() for index in com_area_indexes]

# Create data frame to store information on highest permit fee locations and populate it.
max_fee_loc_df = pd.DataFrame(columns=['community_area','total_fee','latitude','longitude'])                      
i = 0
for j in max_indicies:
    max_fee_loc_df.loc[i]=[permit_df['community_area'][j],
                           permit_df['total_fee'][j],
                           permit_df['latitude'][j],
                           permit_df['longitude'][j]]
    i+=1
    
# Apply community area labels
max_fee_loc_df=max_fee_loc_df.replace({"community_area": community_areas_dict})

max_fee_loc_df

,community_area,total_fee,latitude,longitude
0,South Lawndale,7852.50,41.844514,-87.713143
1,Loop,23103.44,41.884606,-87.626059
2,Morgan Park,2016.12,41.677584,-87.664903
3,Loop,23103.44,41.884606,-87.626059
4,Near West Side,15474.45,41.885433,-87.654528
...,...,...,...,...
964,West Lawn,550.00,41.781135,-87.727571
965,Mount Greenwood,550.00,41.695779,-87.707276
966,Humboldt Park,7272.12,41.895386,-87.712592
967,New City,3390.00,41.805381,-87.642232


In [165]:
per_type_gb=permit_df[['permit_type','processing_time','total_fee']].groupby(['permit_type'], as_index=False)

per_type_sum_df=pd.DataFrame(per_type_gb.sum())

per_type_sum_df

,permit_type,processing_time,total_fee
0,PERMIT - EASY PERMIT PROCESS,463,96519.13
1,PERMIT - ELECTRIC WIRING,200,18100.00
2,PERMIT - ELEVATOR EQUIPMENT,5,15317.00
3,PERMIT - NEW CONSTRUCTION,929,249360.92
4,PERMIT - PORCH CONSTRUCTION,8,200.00
5,PERMIT - REINSTATE REVOKED PMT,0,8891.45
6,PERMIT - RENOVATION/ALTERATION,9223,991022.20
7,PERMIT - SCAFFOLDING,0,2275.00
8,PERMIT - SIGNS,13499,18100.00
9,PERMIT - WRECKING/DEMOLITION,521,7500.00


In [153]:
per_type_ct_df=pd.DataFrame(per_type_gb.count())

per_type_ct_df

,permit_type,processing_time,total_fee
0,PERMIT - EASY PERMIT PROCESS,235,235
1,PERMIT - ELECTRIC WIRING,133,133
2,PERMIT - ELEVATOR EQUIPMENT,20,20
3,PERMIT - NEW CONSTRUCTION,39,39
4,PERMIT - PORCH CONSTRUCTION,1,1
5,PERMIT - REINSTATE REVOKED PMT,12,12
6,PERMIT - RENOVATION/ALTERATION,448,448
7,PERMIT - SCAFFOLDING,7,7
8,PERMIT - SIGNS,59,59
9,PERMIT - WRECKING/DEMOLITION,15,15


In [154]:
per_type_mth_gb=permit_df[['permit_type','start_month','processing_time','total_fee']].groupby([
    'permit_type','start_month'], as_index=False)

per_type_mth_ct_df=pd.DataFrame(per_type_mth_gb.count())

per_type_mth_ct_df[per_type_mth_ct_df['permit_type']=='PERMIT - EASY PERMIT PROCESS']

,permit_type,start_month,processing_time,total_fee
0,PERMIT - EASY PERMIT PROCESS,1,14,14
1,PERMIT - EASY PERMIT PROCESS,2,2,2
2,PERMIT - EASY PERMIT PROCESS,3,13,13
3,PERMIT - EASY PERMIT PROCESS,4,11,11
4,PERMIT - EASY PERMIT PROCESS,5,3,3
5,PERMIT - EASY PERMIT PROCESS,6,11,11
6,PERMIT - EASY PERMIT PROCESS,7,95,95
7,PERMIT - EASY PERMIT PROCESS,8,5,5
8,PERMIT - EASY PERMIT PROCESS,9,5,5
9,PERMIT - EASY PERMIT PROCESS,10,15,15


In [155]:

per_type_mth_ct_df[per_type_mth_ct_df['permit_type']=='PERMIT - RENOVATION/ALTERATION']

,permit_type,start_month,processing_time,total_fee
45,PERMIT - RENOVATION/ALTERATION,1,27,27
46,PERMIT - RENOVATION/ALTERATION,2,22,22
47,PERMIT - RENOVATION/ALTERATION,3,28,28
48,PERMIT - RENOVATION/ALTERATION,4,25,25
49,PERMIT - RENOVATION/ALTERATION,5,28,28
50,PERMIT - RENOVATION/ALTERATION,6,47,47
51,PERMIT - RENOVATION/ALTERATION,7,112,112
52,PERMIT - RENOVATION/ALTERATION,8,39,39
53,PERMIT - RENOVATION/ALTERATION,9,23,23
54,PERMIT - RENOVATION/ALTERATION,10,38,38
